<a href="https://www.kaggle.com/code/matinmahmoudi/pandas-fun-problems-dataframe?scriptVersionId=180218284" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🌿 Pandas Mastery Series - DataFrame

Welcome to the Pandas Mastery Series! In this notebook, we will dive deep into the core structure of pandas: the DataFrame. Our goal is to cover every aspect of DataFrames, providing detailed explanations, practical examples, and fun challenges to enhance your learning experience. Let's get started on our journey to mastering DataFrames!

## Table of Contents

1. **Introduction to DataFrames**
    - What is a DataFrame?
    - Key Features of DataFrames


2. **DataFrame Creation**
    - From Dictionary
    - From List of Lists
    - From List of Dictionaries
    - From Another DataFrame


3. **DataFrame Inspection**
    - DataFrame.info()
    - DataFrame.shape
    - DataFrame.axes
    - DataFrame.size


4. **To And From CSV**
    - DataFrame.to_csv()
    - pandas.read_csv()


5. **Basic Indexing**
    - Indexing Columns
    - Indexing Rows
    - Boolean Indexing


6. **Basic Operations**
    - Inserting Columns
    - Updating Values
    - Removing Columns
    - Changing Column Names


7. **DataFrame.apply()**
    - apply() with Function Arguments


8. **Merging DataFrames**
    - Inner Join
    - Left Join
    - Right Join
    - Outer Join
    - Anti-Join Methods


9. **Aggregation**


10. **Group By**
    - GroupBy Aggregate
    - Renaming Output Columns
    - GroupBy Transform


11. **Fun Challenges**
    - Challenge 1: The Mischievous Data Entry
    - Challenge 2: The Lost Column
    - Challenge 3: The Data Detective
    - Challenge 4: The Aggregation Adventure
    - Challenge 5: The Plotting Puzzle


### Ready for the Ultimate Challenge?

Once you've completed all the notebooks in the Pandas Mastery Series, you'll be ready to tackle the final challenge: [Pandas Mastery Series - Ultimate Challenge](https://www.kaggle.com/code/matinmahmoudi/pandas-mastery-series-ultimate-challenge). This ultimate challenge will put your pandas skills to the test and ensure you're truly a pandas master.

Let's get started and become pandas DataFrame masters!


## 1. Introduction to DataFrames

A DataFrame is a two-dimensional, size-mutable, and heterogeneous tabular data structure with labeled axes (rows and columns) in pandas. It is similar to a spreadsheet or SQL table, and it is one of the most commonly used objects for data manipulation and analysis in pandas.

### What is a DataFrame?

A DataFrame consists of rows and columns:
- **Rows**: Represent individual records or observations.
- **Columns**: Represent different variables or features of the data.

### Key Features of DataFrames
- **Heterogeneous data**: Each column can contain data of different types (integers, floats, strings, etc.).
- **Labeled axes**: Both rows and columns have labels.
- **Size-mutable**: DataFrames can be expanded or contracted.


In [1]:
# Import pandas library
import pandas as pd

# Creating a DataFrame from a dictionary
# Define a dictionary with keys as column names and values as lists of column data
data_dict = {
    'Name': ['Frodo', 'Sam', 'Gandalf', 'Aragorn', 'Legolas'],
    'Race': ['Hobbit', 'Hobbit', 'Wizard', 'Human', 'Elf'],
    'Age': [50, 38, 2019, 87, 2931],
    'Role': ['Ring-bearer', 'Gardener', 'Wizard', 'King', 'Archer']
}
# Create a DataFrame using the dictionary
df_dict = pd.DataFrame(data_dict)
# Print the DataFrame
print("DataFrame created from a dictionary:\n", df_dict)


DataFrame created from a dictionary:
       Name    Race   Age         Role
0    Frodo  Hobbit    50  Ring-bearer
1      Sam  Hobbit    38     Gardener
2  Gandalf  Wizard  2019       Wizard
3  Aragorn   Human    87         King
4  Legolas     Elf  2931       Archer
